In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
# load expression data and labels
X = pd.read_csv("../../data/RNAseq_with_HGNC_symbols.csv", index_col=0)
labels_df = pd.read_csv("../../data/labels.csv", index_col=0)
y = labels_df["Class"].values

# same CV as before
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [8]:
# load variance filtered genes
var_k = 200
var_df = pd.read_csv(f"../../Output/variance_filtering_results/variance_top_genes_k{var_k}.csv")

# dict for fold index -> list of genes
variance_genes = {
    fold_idx + 1: var_df[f"fold_{fold_idx+1}"].dropna().tolist()
    for fold_idx in range(5)
}

In [9]:
# load mutual information filtered genes
mi_genes = {}
for fold in range(1, 6):
    genes = pd.read_csv(
        f"../../Output/mutual_information_results/selected_features_fold_{fold}.csv"
    ).iloc[:, 0].dropna().tolist()
    mi_genes[fold] = genes

In [10]:
# load LASSO filtered genes
l1_genes = {}
for fold in range(1, 6):
    genes = pd.read_csv(
        f"../../Output/l1_results/selected_genes_fold_{fold}.csv"
    ).iloc[:, 0].dropna().tolist()
    l1_genes[fold] = genes

In [13]:
# dicts for storing results
methods = {
    f"variance_filtering": variance_genes,
    "mutual_information": mi_genes,
    "l1": l1_genes,
}

In [15]:
# random forest for each method and fold
results = []

for method_name, fold_to_genes in methods.items():
    fold_accuracies = []

    for fold_idx, (train_idx, test_idx) in enumerate(cv.split(X, y), start=1):
        genes = fold_to_genes[fold_idx]
        # ensure genes are in the dataset
        genes = [g for g in genes if g in X.columns]

        X_train_sel = X.iloc[train_idx][genes]
        X_test_sel  = X.iloc[test_idx][genes]
        y_train = y[train_idx]
        y_test  = y[test_idx]

        rf = RandomForestClassifier(
            n_estimators=500,
            max_depth=None,
            n_jobs=-1,
            random_state=42,
        )

        rf.fit(X_train_sel, y_train)
        y_pred = rf.predict(X_test_sel)
        acc = accuracy_score(y_test, y_pred)
        fold_accuracies.append(acc)
        y_prob = rf.predict_proba(X_test_sel)

        pd.DataFrame(y_test).to_csv(f"../../Output/{method_name}_results/Evaluation_Output/ytrue_{method_name}_fold_{fold_idx}.csv", index=False)
        pd.DataFrame(y_pred).to_csv(f"../../Output/{method_name}_results/Evaluation_Output/ypred_{method_name}_fold_{fold_idx}.csv", index=False)
        pd.DataFrame(y_prob).to_csv(f"../../Output/{method_name}_results/Evaluation_Output/yprob_{method_name}_fold_{fold_idx}.csv", index=False)

        print(f"{method_name} | Fold {fold_idx}: {len(genes)} genes, acc = {acc:.3f}")

    mean_acc = np.mean(fold_accuracies)
    std_acc = np.std(fold_accuracies)
    print(f"\n{method_name}: mean acc = {mean_acc:.3f} ± {std_acc:.3f}\n")

    results.append({
        "method": method_name,
        "mean_accuracy": mean_acc,
        "std_accuracy": std_acc,
    })

# save summary table
results_df = pd.DataFrame(results)
print(results_df)

variance_filtering | Fold 1: 200 genes, acc = 1.000
variance_filtering | Fold 2: 200 genes, acc = 1.000
variance_filtering | Fold 3: 200 genes, acc = 0.994
variance_filtering | Fold 4: 200 genes, acc = 1.000
variance_filtering | Fold 5: 200 genes, acc = 0.988

variance_filtering: mean acc = 0.996 ± 0.005

mutual_information | Fold 1: 200 genes, acc = 1.000
mutual_information | Fold 2: 200 genes, acc = 1.000
mutual_information | Fold 3: 200 genes, acc = 0.994
mutual_information | Fold 4: 200 genes, acc = 1.000
mutual_information | Fold 5: 200 genes, acc = 0.994

mutual_information: mean acc = 0.997 ± 0.003

l1 | Fold 1: 211 genes, acc = 1.000
l1 | Fold 2: 200 genes, acc = 1.000
l1 | Fold 3: 205 genes, acc = 0.994
l1 | Fold 4: 202 genes, acc = 1.000
l1 | Fold 5: 192 genes, acc = 0.994

l1: mean acc = 0.997 ± 0.003

               method  mean_accuracy  std_accuracy
0  variance_filtering        0.99625      0.005000
1  mutual_information        0.99750      0.003062
2                  l1 

In [ ]:
# shuffle labels and repeat
y_shuffled = np.random.permutation(y)

results = []

for method_name, fold_to_genes in methods.items():
    fold_accuracies = []

    for fold_idx, (train_idx, test_idx) in enumerate(cv.split(X, y_shuffled), start=1):
        genes = fold_to_genes[fold_idx]
        genes = [g for g in genes if g in X.columns]

        X_train_sel = X.iloc[train_idx][genes]
        X_test_sel  = X.iloc[test_idx][genes]

        y_train = y_shuffled[train_idx]
        y_test  = y_shuffled[test_idx]

        rf = RandomForestClassifier(
            n_estimators=500,
            max_depth=None,
            n_jobs=-1,
            random_state=42,
        )

        rf.fit(X_train_sel, y_train)
        y_pred = rf.predict(X_test_sel)
        acc = accuracy_score(y_test, y_pred)
        fold_accuracies.append(acc)

        print(f"{method_name} | Fold {fold_idx}: {len(genes)} genes, acc = {acc:.3f}")

    mean_acc = np.mean(fold_accuracies)
    std_acc = np.std(fold_accuracies)
    print(f"\n{method_name}: mean acc = {mean_acc:.3f} ± {std_acc:.3f}\n")

    results.append({
        "method": method_name,
        "mean_accuracy": mean_acc,
        "std_accuracy": std_acc,
    })

results_df = pd.DataFrame(results)
print(results_df)

The above results (running the model on randomized labels) confirm that there is NO major leakage in our pipeline.

In [18]:
# run RF on full dataset 
baseline_accs = []

for fold_idx, (train_idx, test_idx) in enumerate(cv.split(X, y), start=1):
    X_train = X.iloc[train_idx]
    X_test  = X.iloc[test_idx]
    y_train = y[train_idx]
    y_test  = y[test_idx]

    rf = RandomForestClassifier(
        n_estimators=500,
        max_depth=None,
        n_jobs=-1,
        random_state=42
    )

    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    y_prob = rf.predict_proba(X_test)
    baseline_accs.append(acc)
    pd.DataFrame(y_test).to_csv(f"../../Output/original_dataset_results/Evaluation_Output/ytrue_og_fold_{fold_idx}.csv", index=False)
    pd.DataFrame(y_pred).to_csv(f"../../Output/original_dataset_results/Evaluation_Output/ypred_og_fold_{fold_idx}.csv", index=False)
    pd.DataFrame(y_prob).to_csv(f"../../Output/original_dataset_results/Evaluation_Output/yprob_og_fold_{fold_idx}.csv", index=False)

    print(f"Baseline | Fold {fold_idx}: acc = {acc:.3f}")

print(f"\nBaseline RF mean acc = {np.mean(baseline_accs):.3f} ± {np.std(baseline_accs):.3f}")

Baseline | Fold 1: acc = 1.000
Baseline | Fold 2: acc = 1.000
Baseline | Fold 3: acc = 0.994
Baseline | Fold 4: acc = 1.000
Baseline | Fold 5: acc = 0.988

Baseline RF mean acc = 0.996 ± 0.005
